# Note on some columns meaning:

In the calls log:

    - call types: 1:incoming, 2:outgoing, 3:missed
    - is_read: whether the user has interact with the entry
    
In the contacts log:

    - times_contacted: just count of call for the specific contact
    
In the sms logs:
    
    - sms_type: 0:all, 1:inbox, 2:sent, 3:draft, 4:outbox, 5:failed, 6:queued

In [195]:
import pandas as pd
import numpy as np
import glob

from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_from_csv = pd.read_csv('../logs/users.csv')
df_from_json = pd.read_json('../logs/users.json')

In [15]:
df_from_csv.head(3)

,user_id,disbursed_at,loan_status
0,1,2017-03-02T00:00:00.000Z,repaid
1,2,2017-03-01T00:00:00.000Z,defaulted
2,3,2017-03-02T00:00:00.000Z,repaid


In [16]:
df_from_json.head(3)

,disbursed_at,loan_status,user_id
0,2017-03-02,repaid,1
1,2017-03-01,defaulted,2
2,2017-03-02,repaid,3


In [9]:
Counter(df_from_json.disbursed_at)

Counter({Timestamp('2017-03-01 00:00:00'): 67,
         Timestamp('2017-03-02 00:00:00'): 333})

In [10]:
Counter(df_from_csv.disbursed_at)

Counter({'2017-03-01T00:00:00.000Z': 67, '2017-03-02T00:00:00.000Z': 333})

In [14]:
sum(df_from_csv.user_id == df_from_json.user_id), sum(df_from_csv.loan_status == df_from_json.loan_status)

(400, 400)

Looks like the users.csv and users.json contain the same data of the load status and disbursment time of users with id from 1-400, so I will go ahead to use the users.csv

In [94]:
Counter(df_from_csv.loan_status)

Counter({'defaulted': 200, 'repaid': 200})

# Looking at the data of user 164

In [203]:
uid = 164  # picked this user because it has 2 devices
devices = glob.glob('../logs/user-%d/*' %uid)
devices_data = [glob.glob(i+'/*.txt') for i in devices]
devices_data

[['../logs/user-164/device-1/collated_call_log.txt',
  '../logs/user-164/device-1/collated_contact_list.txt',
  '../logs/user-164/device-1/collated_sms_log.txt'],
 ['../logs/user-164/device-2/collated_call_log.txt',
  '../logs/user-164/device-2/collated_contact_list.txt',
  '../logs/user-164/device-2/collated_sms_log.txt']]

In [154]:
df_from_csv[df_from_csv['user_id'] == 164]

,user_id,disbursed_at,loan_status,number_of_devices,repaid
163,164,2017-03-02T00:00:00.000Z,repaid,2,True


# Device 1 of uid 164

In [204]:
dv1_call = pd.read_json(devices_data[0][0])
dv1_call.head(5)

,cached_name,call_type,country_iso,data_usage,datetime,duration,features_video,geocoded_location,is_read,item_id,phone_number
0,Mum,2,KE,0,2017-04-26 17:27:26.907,201,False,Kenya,False,31,710461743
1,Ma Reu,2,KE,0,2017-04-26 17:22:56.508,172,False,Kenya,False,30,254650162978
2,Akiteles,2,KE,0,2017-04-26 17:22:01.934,0,False,Kenya,False,29,718742916
3,Akiteles,2,KE,0,2017-04-26 17:12:04.462,0,False,Kenya,False,28,718742916
4,Akiteles,2,KE,0,2017-04-26 17:10:03.791,0,False,Kenya,False,27,718742916


In [158]:
Counter(dv1_call.cached_name)

Counter({nan: 327,
         u'*128*0772031155': 2,
         u'0790 659030': 79,
         u'128772031155': 1,
         u'AMENELE G': 2,
         u'ARYNE K': 1,
         u'Abu': 1,
         u'Achichi': 1,
         u'Akine': 2,
         u'Akiteles': 5,
         u'Aledi': 28,
         u'Alex!': 7,
         u'Alex-aitel': 1,
         u'Alicia': 3,
         u'Amnyatap!': 2,
         u'Andrew Kapelo': 12,
         u'Anyik!': 3,
         u'Apostle Ben': 2,
         u'Arot': 4,
         u'Arotic': 1,
         u'Arshley': 4,
         u'Ashley!': 1,
         u'Atabo Emar': 16,
         u'Atash': 19,
         u'Athkon': 1,
         u"BA'ryan": 1,
         u'BARNABAS K': 1,
         u'BETTY L': 2,
         u'Ba Chero': 1,
         u'Baba Ipae': 7,
         u'Baba!': 2,
         u'Babaikimat': 1,
         u'Becky': 4,
         u'Benja': 2,
         u'Benny': 1,
         u'Bentar siz': 5,
         u'Bony!': 2,
         u'Brian-ikuku': 2,
         u'Bro Davi': 3,
         u'Bro Eli': 4,
         u'Bro

In [157]:
print Counter(dv1_call.features_video)
print Counter(dv1_call.is_read)

Counter({False: 2098})
Counter({False: 1343, True: 755})


In [205]:
dv1_call['datetime'] = pd.to_datetime(dv1_call['datetime'])
min(dv1_call['datetime']), max(dv1_call['datetime'])

(Timestamp('2017-02-16 09:52:19.049000'),
 Timestamp('2017-04-26 17:27:26.907000'))

In [206]:
np.mean(dv1_call.duration)

44.785033365109626

In [110]:
dv1_contact = pd.read_json(devices_data[0][1])
dv1_contact.head(3)

,display_name,item_id,last_time_contacted,phone_numbers,photo_id,times_contacted
0,Akiteles,1271,1493227362262,"[{u'item_id': 2595, u'normalized_phone_number'...",NaN,6
1,Robert Ekeno,1272,0,"[{u'item_id': 2546, u'normalized_phone_number'...",NaN,0
2,Bentar siz,1273,1493225220050,"[{u'item_id': 5485, u'normalized_phone_number'...",NaN,12


In [131]:
for i in dv1_contact.phone_numbers.head(3).values:
    print i

[{u'item_id': 2595, u'normalized_phone_number': u'+254054247283', u'phone_number': u'0718742916'}]
[{u'item_id': 2546, u'normalized_phone_number': u'+254711346911', u'phone_number': u'+254711346911'}]
[{u'item_id': 5485, u'normalized_phone_number': u'+254323976446', u'phone_number': u'0707379138'}]


In [177]:
dv1_sms = pd.read_json(devices_data[0][2])
dv1_sms.head(3)

,contact_id,datetime,item_id,message_body,sms_address,sms_type,thread_id
0,0,2017-04-10 19:23:38.183,28841,Congratulations! You have received 60.00 KSH ...,Safaricom,1,4034
1,0,2017-04-10 19:12:06.127,28840,I tried calling you at 21:29 on 10Apr Please ...,0722252112,1,3963
2,0,2017-04-10 18:55:14.980,28839,Huku njeve kaka,+254068542642,1,3993


In [159]:
Counter(dv1_sms.sms_type)

Counter({1: 2231, 2: 1971})

In [178]:
Counter(dv1_sms.sms_address)

Counter({u'+254018678736': 8,
         u'+254019610769': 2,
         u'+254040858930': 2,
         u'+254064603459': 130,
         u'+254068542642': 60,
         u'+254094120967': 3,
         u'+254114737896': 90,
         u'+254143913710': 4,
         u'+254187686411': 12,
         u'+254239703771': 3,
         u'+254246932653': 3,
         u'+254275145453': 2,
         u'+254294618681': 2,
         u'+254320142173': 20,
         u'+254339916286': 1,
         u'+254462379307': 8,
         u'+254498363441': 2,
         u'+254498932642': 230,
         u'+254525302683': 3,
         u'+254526557522': 4,
         u'+254554142237': 3,
         u'+254602353546': 2,
         u'+254609264863': 20,
         u'+254623639289': 1,
         u'+254659306671': 2,
         u'+254673666459': 6,
         u'+254699736129': 3,
         u'+254712306481': 12,
         u'+254718270610': 2,
         u'+254745852644': 64,
         u'+254757189128': 140,
         u'+254761014110': 8,
         u'+254808049297': 

# Device 2 of uid 164

In [116]:
dv2_call = pd.read_json(devices_data[1][0])
dv2_call.head(3)

,cached_name,call_type,country_iso,data_usage,datetime,duration,features_video,geocoded_location,is_read,item_id,phone_number
0,Honey!,2,KE,0,2017-05-22 10:47:24.679,0,False,Kenya,False,6781,+254458302406
1,Honey!,2,KE,0,2017-05-22 10:47:16.054,0,False,Kenya,False,6780,+254458302406
2,Robat-platnum,3,KE,0,2017-05-22 06:55:05.352,0,False,Kenya,True,6779,0718902128


In [149]:
dv2_call['datetime'] = pd.to_datetime(dv2_call['datetime'])
min(dv2_call['datetime']), max(dv2_call['datetime'])

(Timestamp('2017-03-12 15:47:14.905000'),
 Timestamp('2017-05-22 10:47:24.679000'))

In [117]:
dv2_contact = pd.read_json(devices_data[1][1])
dv2_contact.head(3)

,display_name,item_id,last_time_contacted,phone_numbers,photo_id,times_contacted
0,Stevo,423,1470677235362,"[{u'item_id': 495222, u'normalized_phone_numbe...",NaN,5
1,Petzz,1188,1486666837411,"[{u'item_id': 495937, u'normalized_phone_numbe...",NaN,28
2,Adhiambo,1514,0,"[{u'item_id': 10620, u'phone_number': u'441'}]",NaN,0


In [121]:
dv2_sms = pd.read_json(devices_data[1][2])
dv2_sms.head(5)

,contact_id,datetime,item_id,message_body,sms_address,sms_type,thread_id
0,0,2017-05-22 10:50:59.602,21381,"Hi Ekuwom, your next Branch repayment of Ksh35...",Branch-Co,1,524
1,0,2017-05-22 10:45:48.087,21380,You have successfully received Ksh 45.00 on P...,Orange,1,561
2,0,2017-05-22 10:45:43.436,21379,Congratulations you have received FREE Holla P...,Orange,1,561
3,0,2017-05-22 10:18:57.252,21378,Hello!,+254458302406,2,225
4,0,2017-05-22 08:20:30.234,21376,You are invited to the Safaricom Deaf Athletic...,SAFARICOM,1,564


In [181]:
dv2_sms[dv2_sms.sms_address == 'Branch-Co'].head(5)

,contact_id,datetime,item_id,message_body,sms_address,sms_type,thread_id
0,0,2017-05-22 10:50:59.602,21381,"Hi Ekuwom, your next Branch repayment of Ksh35...",Branch-Co,1,524
50,0,2017-05-20 10:36:33.079,21330,"Hi Ekuwom, your next Branch repayment of Ksh35...",Branch-Co,1,524
524,0,2017-04-26 19:24:41.516,20027,Your Branch loan of Ksh 3100.0 has been sent! ...,Branch-Co,1,524
800,0,2017-04-03 11:11:28.619,18757,"Hi Ekuwom, your next Branch repayment of Ksh56...",Branch-Co,1,524
1107,0,2017-03-31 11:39:26.447,18427,Your Branch loan of Ksh 2000.0 has been sent! ...,Branch-Co,1,524


In [160]:
Counter(dv2_sms.sms_type)

Counter({1: 5027, 2: 3724, 5: 63})

In [180]:
Counter(dv2_sms.sms_address)

Counter({u'+211863012343': 1,
         u'+254005735214': 2,
         u'+254007510931': 13,
         u'+254008393217': 1,
         u'+254020740405': 6,
         u'+254024948558': 52,
         u'+254025669654': 2,
         u'+254027054132': 2,
         u'+254028991682': 1,
         u'+254032194814': 14,
         u'+254040669378': 51,
         u'+254042255566': 27,
         u'+254047392031': 1,
         u'+254047820042': 25,
         u'+254049403345': 1,
         u'+254049888677': 33,
         u'+254054247283': 8,
         u'+254057552783': 1,
         u'+254060432444': 3,
         u'+254071349735': 32,
         u'+254072684297': 23,
         u'+254078310919': 7,
         u'+254083973428': 1,
         u'+254087134351': 3,
         u'+254092593154': 15,
         u'+254093546874': 18,
         u'+254100705546': 1,
         u'+254106294864': 15,
         u'+254111548588': 1,
         u'+254112189269': 2,
         u'+254117514912': 28,
         u'+254119024677': 27,
         u'+254121570830':

# let's look at data from another user (uid 232)

In [199]:
uid = 232 # default  
devices = glob.glob('../logs/user-%d/*' %uid)
devices_data = [glob.glob(i+'/*.txt') for i in devices]
devices_data

[['../logs/user-232/device-1/collated_call_log.txt',
  '../logs/user-232/device-1/collated_contact_list.txt',
  '../logs/user-232/device-1/collated_sms_log.txt']]

In [165]:
df_from_csv[df_from_csv['user_id'] == 232]

,user_id,disbursed_at,loan_status,number_of_devices,repaid
231,232,2017-03-01T00:00:00.000Z,defaulted,1,False


# Device 1 of uid 220

In [200]:
dv1_call = pd.read_json(devices_data[0][0])
dv1_call.head(3)

,cached_name,call_type,datetime,duration,is_read,item_id,phone_number
0,NaN,1,2017-04-10 19:23:38.183,0,False,51734,Safaricom
1,Ngomongo Machakachue,1,2017-04-10 19:12:06.127,0,False,51733,0722252112
2,Mzee Sagini Majimoto,1,2017-04-10 18:55:14.980,0,False,51732,+254068542642


In [201]:
dv1_call['datetime'] = pd.to_datetime(dv1_call['datetime'])
min(dv1_call['datetime']), max(dv1_call['datetime'])

(Timestamp('2017-01-21 16:40:17.285000'),
 Timestamp('2017-04-10 19:23:38.183000'))

In [202]:
np.mean(dv1_call.duration)

23.537665562913908

In [170]:
Counter(dv1_call.cached_name)

Counter({nan: 788,
         u'+254719833858': 6,
         u'0727777760': 17,
         u'Abel': 111,
         u'Abough': 1,
         u'Aesha': 10,
         u'Anta': 3,
         u'Baba Sumaya': 36,
         u'Babbu': 105,
         u'Bnz.': 3,
         u'Brooohhh': 7,
         u'Clyde': 6,
         u'Dada': 30,
         u'Fadhili': 2,
         u'Free Call': 13,
         u'Halima': 5,
         u'Hassan': 39,
         u'Herb': 42,
         u'JIMMY JAMAL': 12,
         u'Joan': 1,
         u'Jude': 2,
         u'Jumbe': 1,
         u'Kimani Bildad': 27,
         u'Kirago': 39,
         u'Ma Khadija': 369,
         u'Maahad': 5,
         u'Makhanu': 7,
         u'Meddy': 2,
         u'Mgustavo': 77,
         u'Muha': 35,
         u'Mzee Sagini Majimoto': 73,
         u'Nancy Mafuta': 1,
         u'Ngomongo Machakachue': 349,
         u'Omar12': 15,
         u'Panda': 2,
         u'Peter': 136,
         u'Peterr': 3,
         u'Qassim2': 6,
         u'Sportpesa': 9,
         u'Tonny': 7,
     

In [172]:
dv1_contact = pd.read_json(devices_data[0][1])
dv1_contact.head(3)

,display_name,item_id,last_time_contacted,phone_numbers,photo_id,times_contacted
0,Baba,2056,1461738812364,"[{u'item_id': 2467, u'normalized_phone_number'...",NaN,16
1,Mama Suleiman,6372,0,NaN,NaN,0
2,Muha,6941,1489142087743,"[{u'item_id': 12882, u'normalized_phone_number...",NaN,162


In [182]:
dv1_sms = pd.read_json(devices_data[0][2])
dv1_sms.head(3)

,contact_id,datetime,item_id,message_body,sms_address,sms_type,thread_id
0,0,2017-04-10 19:23:38.183,28841,Congratulations! You have received 60.00 KSH ...,Safaricom,1,4034
1,0,2017-04-10 19:12:06.127,28840,I tried calling you at 21:29 on 10Apr Please ...,0722252112,1,3963
2,0,2017-04-10 18:55:14.980,28839,Huku njeve kaka,+254068542642,1,3993


In [174]:
Counter(dv1_sms.sms_type)

Counter({1: 1133, 2: 725, 5: 9})

In [175]:
Counter(dv1_sms.sms_address)

Counter({u'+254018678736': 8,
         u'+254019610769': 2,
         u'+254040858930': 2,
         u'+254064603459': 130,
         u'+254068542642': 60,
         u'+254094120967': 3,
         u'+254114737896': 90,
         u'+254143913710': 4,
         u'+254187686411': 12,
         u'+254239703771': 3,
         u'+254246932653': 3,
         u'+254275145453': 2,
         u'+254294618681': 2,
         u'+254320142173': 20,
         u'+254339916286': 1,
         u'+254462379307': 8,
         u'+254498363441': 2,
         u'+254498932642': 230,
         u'+254525302683': 3,
         u'+254526557522': 4,
         u'+254554142237': 3,
         u'+254602353546': 2,
         u'+254609264863': 20,
         u'+254623639289': 1,
         u'+254659306671': 2,
         u'+254673666459': 6,
         u'+254699736129': 3,
         u'+254712306481': 12,
         u'+254718270610': 2,
         u'+254745852644': 64,
         u'+254757189128': 140,
         u'+254761014110': 8,
         u'+254808049297': 

In [193]:
dv1_sms[dv1_sms.sms_address == 'Branch-Co'].head(5)

,contact_id,datetime,item_id,message_body,sms_address,sms_type,thread_id
41,0,2017-04-09 11:10:46.416,28798,"Hi Abubakar Siddique, Branch reports any non-p...",Branch-Co,1,4030
74,0,2017-04-06 10:15:43.489,28763,"Hi Abubakar Siddique, you are very late on you...",Branch-Co,1,4030
93,0,2017-04-03 10:23:39.639,28744,"Hi Abubakar Siddique, you are very late on you...",Branch-Co,1,4030
134,0,2017-03-31 10:41:28.476,28700,"Hi Abubakar Siddique, you are very late on you...",Branch-Co,1,4030
149,0,2017-03-28 11:06:58.839,28682,"Hi Abubakar Siddique, you are very late on you...",Branch-Co,1,4030
